In [2]:
# Run in python console
import nltk; nltk.download('stopwords')

import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/paperspace/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [3]:
jobs_df = pd.read_csv('~/data/jobs_df.csv')
duties = jobs_df.job_duties
data = duties

In [5]:
# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [7]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
#Add Stop Word Extensions
stop_words.extend(['may', 'los', 'angeles', 'work', 'use'])


# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

# Initialize spacy 'en' model, keeping only tagger component (for efficiency)
# python3 -m spacy download en
nlp = spacy.load('en', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

In [10]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

In [11]:
#Params
num_topics = 10
random_state = 100
chunksize = 100
passes = 10
update_every = 1
alpha = 'auto'

In [12]:
# LDA Model creation
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=num_topics, 
                                           random_state=random_state,
                                           update_every=update_every,
                                           chunksize=chunksize,
                                           passes=passes,
                                           alpha='auto',
                                           per_word_topics=True)

In [12]:
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.045*"water" + 0.031*"service" + 0.018*"environmental" + 0.016*"customer" '
  '+ 0.015*"perform" + 0.013*"document" + 0.013*"information" + 0.011*"power" '
  '+ 0.011*"report" + 0.011*"account"'),
 (1,
  '0.028*"maintain" + 0.026*"apply" + 0.026*"build" + 0.025*"supervisory" + '
  '0.025*"technique" + 0.025*"sound" + 0.024*"effective" + 0.024*"principle" + '
  '0.023*"employment" + 0.023*"opportunity"'),
 (2,
  '0.015*"traffic" + 0.014*"parking" + 0.012*"inspection" + '
  '0.012*"installation" + 0.011*"inspect" + 0.011*"material" + 0.011*"control" '
  '+ 0.011*"building" + 0.011*"make" + 0.010*"city"'),
 (3,
  '0.017*"city" + 0.014*"assign" + 0.013*"license" + 0.013*"permit" + '
  '0.012*"property" + 0.012*"conduct" + 0.011*"investigator" + '
  '0.011*"treatment" + 0.011*"tax" + 0.010*"require"'),
 (4,
  '0.067*"equipment" + 0.041*"electrical" + 0.041*"maintenance" + '
  '0.041*"repair" + 0.031*"system" + 0.020*"mechanical" + 0.017*"perform" + '
  '0.015*"operator" + 0.015*"el

In [13]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.323200  0.009374       1        1  36.717896
8      0.008740 -0.195155       2        1  13.931984
2      0.024734  0.127587       3        1  12.121288
0      0.046184 -0.067091       4        1   9.348281
4      0.014221  0.312006       5        1   9.025124
9      0.092731 -0.076075       6        1   7.036406
6     -0.030378 -0.080250       7        1   4.890402
3     -0.098906 -0.071827       8        1   4.347981
5     -0.199473  0.043682       9        1   1.600015
7     -0.181054 -0.002251      10        1   0.980622, topic_info=     Category        Freq           Term       Total  loglift  logprob
88    Default  241.000000      equipment  241.000000  30.0000  30.0000
52    Default  239.000000    maintenance  239.000000  29.0000  29.0000
78    Default  165.000000          water  165.000000  28.0000  28.0000
64    Default  161.000000         repair  161.000000  27.0000  27.0000
43    Default  128.000000     electrical  128.000000  26.0000  26.0000
4     Default  326.000000          build  326.000000  25.0000  25.0000
31    Default  314.000000    supervisory  314.000000  24.0000  24.0000
28    Default  308.000000          sound  308.000000  23.0000  23.0000
9     Default  306.000000      effective  306.000000  22.0000  22.0000
24    Default  306.000000      principle  306.000000  21.0000  21.0000
16    Default  288.000000        fulfill  288.000000  20.0000  20.0000
32    Default  317.000000      technique  317.000000  19.0000  19.0000
22    Default  278.000000           plan  278.000000  18.0000  18.0000
18    Default  389.000000       maintain  389.000000  17.0000  17.0000
15    Default  271.000000          force  271.000000  16.0000  16.0000
118   Default   58.000000  communication   58.000000  15.0000  15.0000
0     Default  339.000000          apply  339.000000  14.0000  14.0000
73    Default  243.000000         system  243.000000  13.0000  13.0000
95    Default  322.000000        perform  322.000000  12.0000  12.0000
61    Default  205.000000        program  205.000000  11.0000  11.0000
38    Default  258.000000         direct  258.000000  10.0000  10.0000
20    Default  303.000000    opportunity  303.000000   9.0000   9.0000
11    Default  336.000000     employment  336.000000   8.0000   8.0000
13    Default  308.000000          equal  308.000000   7.0000   7.0000
205   Default   69.000000         police   69.000000   6.0000   6.0000
68    Default  235.000000        service  235.000000   5.0000   5.0000
208   Default  130.000000         public  130.000000   4.0000   4.0000
54    Default  140.000000     management  140.000000   3.0000   3.0000
10    Default  246.000000       employee  246.000000   2.0000   2.0000
5     Default  313.000000           city  313.000000   1.0000   1.0000
...       ...         ...            ...         ...      ...      ...
1679  Topic10    4.269313         splice    5.058792   4.4551  -4.3646
1238  Topic10    4.002142        trouble    4.791552   4.4447  -4.4292
2074  Topic10    3.686706           tire    4.476092   4.4307  -4.5113
1350  Topic10   11.945129     recreation   15.137221   4.3879  -3.3357
1675  Topic10    2.659213         copper    3.448525   4.3648  -4.8380
2795  Topic10    2.347765            wan    3.137114   4.3349  -4.9626
2794  Topic10    2.347765            lan    3.137114   4.3349  -4.9626
1745  Topic10    2.086271           main    2.877097   4.3033  -5.0807
1948  Topic10    2.019903      microwave    2.809220   4.2949  -5.1130
2072  Topic10    2.018636           road    2.808077   4.2947  -5.1136
1673  Topic10    1.893291        carrier    2.682603   4.2763  -5.1777
1677  Topic10    1.893291           pair    2.682603   4.2763  -5.1777
1680  Topic10    1.893291      telemetry    2.682603   4.2763  -5.1777
2720  Topic10    1.581941        message    2.371378   4.2199  -5.3574
2684  Topic10    1.580700    containment   

In [13]:
# Download File: http://mallet.cs.umass.edu/dist/mallet-2.0.8.zip
mallet_path = '~/data/mallet-2.0.8/bin/mallet' # update this path

In [14]:
#Run LDA Mallet
ldamallet = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)

# Show Topics
pprint(ldamallet.show_topics(formatted=False))

/home/paperspace/.local/lib/python3.6/site-packages/smart_open/smart_open_lib.py:398: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


[(0,
  [('service', 0.06634460547504026),
   ('water', 0.0644122383252818),
   ('include', 0.027375201288244767),
   ('operator', 0.021900161030595812),
   ('customer', 0.01996779388083736),
   ('provide', 0.019001610305958132),
   ('field', 0.01610305958132045),
   ('collection', 0.014814814814814815),
   ('make', 0.014170692431561997),
   ('treatment', 0.014170692431561997)]),
 (1,
  [('engage', 0.09367602299628001),
   ('maintain', 0.0892796753466351),
   ('employee', 0.08251606357795063),
   ('supervise', 0.05410889414947582),
   ('evaluate', 0.049036185322962465),
   ('senior', 0.044978018261751775),
   ('build', 0.043287115319580655),
   ('supervisor', 0.04024349002367264),
   ('group', 0.0368616841393304),
   ('technique', 0.032127155901251266)]),
 (2,
  [('relate', 0.041666666666666664),
   ('building', 0.032),
   ('public', 0.030333333333333334),
   ('safety', 0.026),
   ('investigation', 0.023666666666666666),
   ('police', 0.022333333333333334),
   ('officer', 0.020666666666

In [15]:
# Compute Coherence Score
coherence_model_ldamallet = CoherenceModel(model=ldamallet, texts=data_lemmatized, dictionary=id2word, coherence='c_v')
coherence_ldamallet = coherence_model_ldamallet.get_coherence()
print('\nCoherence Score: ', coherence_ldamallet)


Coherence Score:  0.5331455796772641


In [16]:
def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    """
    Compute c_v coherence for various number of topics

    Parameters:
    ----------
    dictionary : Gensim dictionary
    corpus : Gensim corpus
    texts : List of input texts
    limit : Max num of topics

    Returns:
    -------
    model_list : List of LDA topic models
    coherence_values : Coherence values corresponding to the LDA model with respective number of topics
    """
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
        model_list.append(model)
        coherencemodel = CoherenceModel(model=model, texts=texts, dictionary=dictionary, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [18]:
###For the LDA Model
start = 2
limit = 40
step = 6
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)

2019-06-16 14:47:53,307 : ERROR : Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/home/paperspace/.local/lib/python3.6/site-packages/IPython/core/interactiveshell.py", line 3296, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-18-44e4489dc321>", line 5, in <module>
    model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)
  File "<ipython-input-17-f6c10e56a163>", line 20, in compute_coherence_values
    model = gensim.models.wrappers.LdaMallet(mallet_path, corpus=corpus, num_topics=num_topics, id2word=id2word)
  File "/home/paperspace/.local/lib/python3.6/site-packages/gensim/models/wrappers/ldamallet.py", line 132, in __init__
    self.train(corpus)
  File "/home/paperspace/.local/lib/python3.6/site-packages/gensim/models/wrappers/ldamallet.py", line 285, in train
    check_output(args=cmd, shell=True)
  File "/home/paperspace/.local/lib/python3.6/site-packages/gensim/utils.py", line 19

Exception in thread QueueManagerThread:
Traceback (most recent call last):
  File "/usr/lib/python3.6/threading.py", line 916, in _bootstrap_inner
    self.run()
  File "/usr/lib/python3.6/threading.py", line 864, in run
    self._target(*self._args, **self._kwargs)
  File "/home/paperspace/.local/lib/python3.6/site-packages/joblib/externals/loky/process_executor.py", line 674, in _queue_management_worker
    recursive_terminate(p)
  File "/home/paperspace/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/utils.py", line 28, in recursive_terminate
    _recursive_terminate_without_psutil(process)
  File "/home/paperspace/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/utils.py", line 53, in _recursive_terminate_without_psutil
    _recursive_terminate(process.pid)
  File "/home/paperspace/.local/lib/python3.6/site-packages/joblib/externals/loky/backend/utils.py", line 94, in _recursive_terminate
    stderr=None
  File "/usr/lib/python3.6/subprocess.py", li

KeyboardInterrupt: 

In [ ]:
limit=40; start=2; step=6;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [ ]:
start = 2
limit = 20
step = 1

In [ ]:
model_list, coherence_values = compute_coherence_values(dictionary=id2word, corpus=corpus, texts=data_lemmatized, start=start, limit=limit, step=step)

In [ ]:
limit=limit; start=start; step=step;
x = range(start, limit, step)
plt.plot(x, coherence_values)
plt.xlabel("Num Topics")
plt.ylabel("Coherence score")
plt.legend(("coherence_values"), loc='best')
plt.show()

In [ ]:
for m, cv in zip(x, coherence_values):
    print("Num Topics =", m, " has Coherence Value of", round(cv, 3))

In [17]:
#rerunning with best params

#Params
num_topics = 15

#Run LDA Mallet2
ldamallet2 = gensim.models.wrappers.LdaMallet(
    mallet_path, 
    corpus=corpus, 
    num_topics=num_topics, 
    id2word=id2word)

In [18]:
lda_model3 = gensim.models.wrappers.ldamallet.malletmodel2ldamodel(ldamallet)


#pyLDAvis.enable_notebook()
#vis = pyLDAvis.gensim.prepare(lda_model3, corpus, id2word)
#vis

In [19]:
def format_topics_sentences(ldamodel=lda_model, corpus=corpus, texts=data):
    # Init output
    sent_topics_df = pd.DataFrame()

    # Get main topic in each document
    for i, row in enumerate(ldamodel[corpus]):
        row = sorted(row, key=lambda x: (x[1]), reverse=True)
        # Get the Dominant topic, Perc Contribution and Keywords for each document
        for j, (topic_num, prop_topic) in enumerate(row):
            if j == 0:  # => dominant topic
                wp = ldamodel.show_topic(topic_num)
                topic_keywords = ", ".join([word for word, prop in wp])
                sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
            else:
                break
    sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

    # Add original text to the end of the output
    contents = pd.Series(texts)
    sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
    return(sent_topics_df)


In [20]:
optimal_model = lda_model3

df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=corpus, texts=data)

# Format
df_dominant_topic = df_topic_sents_keywords.reset_index()
df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']

# Show
df_dominant_topic.head(15)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text
0,0,9.0,0.1983,"supervisory, principle, effective, opportunity...",A Landscape Architect oversees and prepares pr...
1,1,8.0,0.2033,"activity, construction, maintenance, operation...","An Electrical Services Manager plans, directs,..."
2,2,5.0,0.2452,"equipment, repair, electrical, control, mechan...",A Wastewater Treatment Mechanic Supervisor ass...
3,3,9.0,0.2131,"supervisory, principle, effective, opportunity...",An Equipment Repair Supervisor supervises the ...
4,4,3.0,0.1652,"system, perform, engineering, design, informat...",A Building Electrical Engineer supervises and ...
5,5,5.0,0.1510,"equipment, repair, electrical, control, mechan...",An Aquarist procures and maintains a variety o...
6,6,3.0,0.2056,"system, perform, engineering, design, informat...",A Land Surveying Assistant sets up and operate...
7,7,2.0,0.3361,"relate, building, public, safety, investigatio...",A Port Police Officer is a sworn peace officer...
8,8,5.0,0.2476,"equipment, repair, electrical, control, mechan...",A Warehouse and Toolroom Worker performs manua...
9,9,7.0,0.7810,"fire, city, department, position, require, can...",/MINIMUM QUALIFICATION Four years of full-time...


In [21]:
import pandas as pd
import numpy as np
import glob
import os
import re

In [12]:
jobs_df = pd.read_csv('~/data/jobs_df.csv')
duties = jobs_df.job_duties

In [13]:
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import CountVectorizer
 
NUM_TOPICS = 10

msw = text.ENGLISH_STOP_WORDS.union(['may', 'los', 'angeles', 'work', 'use','city'])

vectorizer = CountVectorizer(min_df=5, max_df=0.9, 
                             stop_words=msw, lowercase=True, 
                             token_pattern='[a-zA-Z\-][a-zA-Z\-]{2,}')
data_vectorized = vectorizer.fit_transform(duties)

In [14]:
nmf_model = NMF(n_components=NUM_TOPICS)
nmf_1 = nmf_model.fit_transform(data_vectorized)
nmf_1

array([[0.00000000e+00, 4.77466693e-01, 0.00000000e+00, ...,
        0.00000000e+00, 2.66485244e-02, 1.71765795e-01],
       [0.00000000e+00, 4.91824240e-01, 2.98957499e-01, ...,
        0.00000000e+00, 0.00000000e+00, 1.13089844e-01],
       [0.00000000e+00, 5.38567401e-01, 2.89771168e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       ...,
       [2.96369043e-03, 4.72888065e-01, 1.49337548e-01, ...,
        2.39742215e-07, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 5.55759409e-01, 1.85005542e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 4.60767060e-01, 4.06158767e-01, ...,
        0.00000000e+00, 0.00000000e+00, 0.00000000e+00]])

In [15]:
from sklearn import linear_model
from sklearn import model_selection as ms
import warnings
warnings.warn("once")
nmf_model = NMF()

# Use grid search to tune the parameters:

#grid paramters
estimator = nmf_model
scoring = 'r2'
param_grid = {"alpha": np.arange(2**(-5), 2**8, .1),
              "l1_ratio": np.arange(0.0, 1.0, 0.1),
             "n_components": np.arange(4, 40, 1)}
cv = 10

#grid based model
grid = ms.GridSearchCV(estimator = estimator, param_grid = param_grid, \
                    scoring=scoring, cv=cv)

#fit the model
grid.fit(data_vectorized)

/home/paperspace/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: UserWarning: once
  after removing the cwd from sys.path.


TypeError: __call__() missing 1 required positional argument: 'y_true'

In [3]:
def print_topics2(model, vectorizer, top_n=10):
    df = pd.DataFrame()
    for idx, topic in enumerate(model.components_):
        df['idx'] = idx
        df['tuple'] = [(vectorizer.get_feature_names()[i], topic[i]) for i in topic.argsort()[:-top_n - 1:-1]]
    return df

print_topics2(nmf11, tdfidf)

NameError: name 'nmf11' is not defined

In [ ]:
vectorizer.get_feature_names()

In [ ]:
def get_nmf_topics(model, n_top_words):
    
    #the word ids obtained need to be reverse-mapped to the words so we can print the topic names.
    feat_names = vectorizer.get_feature_names()
    
    topics = []
    for i in range(num_topics):
        
        #for each topic, obtain the largest values, and add the words they map to into the dictionary.
        words_ids = model.components_[i].argsort()[:-20 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        topics.append(words)
    return topics

In [ ]:
num_topics = 10
top_lis = get_nmf_topics(nmf_1, -1)
top_lis

In [ ]:
cm = CoherenceModel(topics=top_lis, texts=data_lemmatized, dictionary=id2word, coherence='c_v')

In [ ]:
from sklearn.decomposition import NMF
from sklearn.decomposition.nmf import _beta_divergence 

num_tops = range(4, 40, 1)
top_lis = []
rec_err_lis = []
for top in num_tops:
    nmf = NMF(n_components=top, random_state=0, alpha=.1, l1_ratio=.5).fit(data_vectorized)
    rec_err = nmf.reconstruction_err_
    print("Number of Topics:", top,"|" " Reconstruction error:", rec_err)

In [ ]:
nmf8 = NMF(n_components=8, random_state=0, alpha=.1, l1_ratio=.5).fit(data_vectorized)
print_topics(nmf8, vectorizer)

In [ ]:
nmf9 = NMF(n_components=9, random_state=0, alpha=.1, l1_ratio=.5).fit(data_vectorized)
print_topics(nmf9, vectorizer)

In [ ]:
nmf10 = NMF(n_components=10, random_state=0, alpha=.1, l1_ratio=.5).fit(data_vectorized)
print_topics(nmf10, vectorizer)

In [66]:
nmf11 = NMF(n_components=11, random_state=0, alpha=.1, l1_ratio=.5).fit(data_vectorized)
print_topics(nmf11, vectorizer)

Topic 0:
[('pilot', 1.7157464042999442), ('training', 1.2764916010461995), ('department', 1.2370391273686712), ('time', 1.0127687945777586), ('experience', 0.8162269541869979), ('application', 0.8090314038504095), ('completion', 0.7939650814055297), ('school', 0.7704498533873871), ('examination', 0.7319126428773355), ('trainee', 0.7315023889258918)]
Topic 1:
[('building', 2.4857352304736393), ('maintaining', 2.478471556885737), ('applies', 2.3506838774577394), ('effective', 2.345352775238827), ('techniques', 2.332393365307313), ('sound', 2.310951650842385), ('responsibilities', 2.3041547859075577), ('supervisory', 2.302550273518551), ('principles', 2.2973612148483493), ('equal', 2.2823189287223316)]
Topic 2:
[('equipment', 3.5493345631981605), ('electrical', 2.359335732151503), ('maintenance', 1.677096960070999), ('electric', 1.45482279227944), ('repair', 1.3322137296641339), ('distribution', 1.1141076317650784), ('performs', 1.0314656965102103), ('related', 0.9467798305992168), ('cons

In [22]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
score_lis = []
for text in df_dominant_topic['Text']:
    vs = analyzer.polarity_scores(text)
    score_lis.append(vs)

In [23]:
neg_lis = []
neu_lis = []
pos_lis = []
comp_lis = []
for dic in score_lis:
    neg_lis.append(dic["neg"])
    neu_lis.append(dic["neu"])
    pos_lis.append(dic["pos"])
    comp_lis.append(dic["compound"])
df_dominant_topic['vader_neg'] = neg_lis
df_dominant_topic['vader_neu'] = neu_lis
df_dominant_topic['vader_pos'] = pos_lis
df_dominant_topic['vader_compound'] = comp_lis
df_dominant_topic.head(5)

,Document_No,Dominant_Topic,Topic_Perc_Contrib,Keywords,Text,vader_neg,vader_neu,vader_pos,vader_compound
0,0,9.0,0.1983,"supervisory, principle, effective, opportunity...",A Landscape Architect oversees and prepares pr...,0.000,0.819,0.181,0.8625
1,1,8.0,0.2033,"activity, construction, maintenance, operation...","An Electrical Services Manager plans, directs,...",0.000,0.882,0.118,0.9169
2,2,5.0,0.2452,"equipment, repair, electrical, control, mechan...",A Wastewater Treatment Mechanic Supervisor ass...,0.000,0.872,0.128,0.8625
3,3,9.0,0.2131,"supervisory, principle, effective, opportunity...",An Equipment Repair Supervisor supervises the ...,0.075,0.732,0.194,0.7783
4,4,3.0,0.1652,"system, perform, engineering, design, informat...",A Building Electrical Engineer supervises and ...,0.026,0.869,0.105,0.7351


In [57]:
df_dominant_topic.groupby('Dominant_Topic').mean()

,Document_No,Topic_Perc_Contrib,vader_neg,vader_neu,vader_pos,vader_compound
Dominant_Topic,,,,,,
0.0,322.233333,0.236522,0.016700,0.887267,0.096100,0.532577
1.0,325.655556,0.211508,0.006911,0.861089,0.132011,0.737084
2.0,351.284553,0.224426,0.015350,0.806154,0.178472,0.828171
3.0,327.252747,0.255512,0.026484,0.916363,0.057110,0.226680
4.0,337.741379,0.231338,0.031931,0.877586,0.090517,0.331547
5.0,353.681818,0.232664,0.036939,0.885515,0.077530,0.272768
6.0,364.785714,0.479643,0.075286,0.867500,0.057071,0.268564
7.0,356.678571,0.240768,0.063857,0.848821,0.087321,0.119018
8.0,346.098592,0.233468,0.028127,0.916380,0.055479,0.208977


In [60]:
from sklearn.metrics.pairwise import euclidean_distances
 
def most_similar(x, Z, top_n=5):
    dists = euclidean_distances(x.reshape(1, -1), Z)
    pairs = enumerate(dists[0])
    most_similar = sorted(pairs, key=lambda item: item[1])[:top_n]
    return most_similar

In [ ]:
com_lis = []
for text in nmf_dominant_topic['text']:
    mod = nmf11.transform(vectorizer.transform([text]))[0]
    topp = mod.argmax(axis=0)
    com_lis.append(topp)

In [72]:
nmf_dominant_topic = pd.DataFrame()
nmf_dominant_topic['keywords'] = df_dominant_topic['Keywords']
nmf_dominant_topic['text'] = df_dominant_topic['Text']


nmf_dominant_topic['NMF Topic'] = com_lis
nmf_dominant_topic.head(5)

,keywords,text,NMF Topic
0,"system, construction, plan, engineering, proje...",A Landscape Architect oversees and prepares pr...,1
1,"direct, activity, plan, organize, operation, d...","An Electrical Services Manager plans, directs,...",9
2,"equipment, system, mechanical, repair, mainten...",A Wastewater Treatment Mechanic Supervisor ass...,1
3,"supervisory, apply, engage, maintain, force, b...",An Equipment Repair Supervisor supervises the ...,1
4,"system, construction, plan, engineering, proje...",A Building Electrical Engineer supervises and ...,6


In [75]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
analyzer = SentimentIntensityAnalyzer()
nmfscore_lis = []
for text in nmf_dominant_topic['text']:
    vs = analyzer.polarity_scores(text)
    nmfscore_lis.append(vs)

In [77]:
neg_lis2 = []
neu_lis2 = []
pos_lis2 = []
comp_lis2 = []
for dic in nmfscore_lis:
    neg_lis2.append(dic["neg"])
    neu_lis2.append(dic["neu"])
    pos_lis2.append(dic["pos"])
    comp_lis2.append(dic["compound"])
nmf_dominant_topic['vader_neg'] = neg_lis2
nmf_dominant_topic['vader_neu'] = neu_lis2
nmf_dominant_topic['vader_pos'] = pos_lis2
nmf_dominant_topic['vader_compound'] = comp_lis2
nmf_dominant_topic.head(5)

,keywords,text,NMF Topic,vader_neg,vader_neu,vader_pos,vader_compound
0,"system, construction, plan, engineering, proje...",A Landscape Architect oversees and prepares pr...,1,0.000,0.819,0.181,0.8625
1,"direct, activity, plan, organize, operation, d...","An Electrical Services Manager plans, directs,...",9,0.000,0.882,0.118,0.9169
2,"equipment, system, mechanical, repair, mainten...",A Wastewater Treatment Mechanic Supervisor ass...,1,0.000,0.872,0.128,0.8625
3,"supervisory, apply, engage, maintain, force, b...",An Equipment Repair Supervisor supervises the ...,1,0.075,0.732,0.194,0.7783
4,"system, construction, plan, engineering, proje...",A Building Electrical Engineer supervises and ...,6,0.026,0.869,0.105,0.7351


In [79]:
nmf_dominant_topic.groupby('NMF Topic').mean()

,vader_neg,vader_neu,vader_pos,vader_compound
NMF Topic,,,,
0,0.035000,0.918600,0.046400,0.246910
1,0.014089,0.823232,0.162671,0.814221
2,0.028204,0.922667,0.049118,0.144501
3,0.048017,0.889403,0.062605,0.110141
4,0.030000,0.907500,0.062441,0.188944
5,0.113500,0.854500,0.032000,0.263150
6,0.016722,0.915278,0.067870,0.353446
7,0.032900,0.852750,0.114350,0.565590
8,0.052778,0.890889,0.056111,0.341278


In [58]:
masc = open('masc.txt', 'r')
masc_list = [line.split(',') for line in masc.readlines()]

flat_list = []
for sublist in masc_list:
    for item in sublist:
        flat_list.append(item)
masc.list = [i.replace('\n', "") for i in flat_list]

In [ ]:
fem = open('fem.txt', 'r')
fem_list = [line.split(',') for line in fem.readlines()]

flat_list2 = []
for sublist in fem_list:
    for item in sublist:
        flat_list2.append(item)
fem.list = [i.replace('\n', "") for i in flat_list2]

In [40]:
def Convert(string): 
    li = list(string.split(" ")) 
    return li

In [60]:
['num_masc'] = sum(el in masc.list for el in [Convert(tex) for tex in df_dominant_topic['Text']])
df_dominant_topic.sample(5)

###SOUNDS GREAT DOESN'T WORK

SyntaxError: can't assign to literal (<ipython-input-60-9f188e352c32>, line 1)

In [9]:
jobs_df['requirements'].sample(25)

433    Four years of full-time paid experience with t...
347    1.  Graduation from an accredited four-year co...
424    Four years of full-time paid housing rehabilit...
334    1. Graduation from a high school or the equiva...
465    1. Graduation from an accredited four-year col...
49     A valid T2 (or higher) Water Treatment Operato...
630    1. Current employment with the City of Los Ang...
414    1. One year of full-time paid experience as a ...
96     Successful completion of any combination of tw...
86     1. Graduation from an accredited four-year col...
220    Three years of full-time paid experience condu...
579    Four years of full-time paid experience as a j...
355    1. Two years of full-time paid experience as a...
656    /MINUMUM QUALIFICATIONS\n1. Possess a current ...
517    Two years of full-time paid office clerical ex...
20     1. Graduation from an accredited four-year col...
408    A Bachelor's degree from an accredited four-ye...
155    1. Graduation from an ac